# SPGR

In [1]:
from multiprocessing import set_start_method
set_start_method('forkserver')

In [2]:
import version_control

Last run on cashlab03-deb at 2015-04-09 09:23:09.227989

Phypno Version: 6374c95883a9a9260ede7e2d451890f11c0e744a

Babel==1.3, Cython==0.22, Jinja2==2.7.3, Markdown==2.6.1, MarkupSafe==0.23, Pillow==2.8.1, PyOpenGL==3.1.0, Pygments==2.0.2, Sphinx==1.3.1, alabaster==0.7.3, bash-kernel==0.3, beautifulsoup4==4.3.2, biopython==1.65, certifi==14.05.14, coverage==3.7.1, decorator==3.4.2, docutils==0.12, easydev==0.8.3, ipython==3.1.0, jedi==0.8.1-final0, jsonschema==2.4.0, matplotlib==1.4.3, memory-profiler==0.32, mistune==0.5.1, networkx==1.9.1, nibabel==2.0.0, nitime==0.5, nltk==3.0.2, nose==1.3.6, numpy==1.9.2, numpydoc==0.5, ordereddict==1.1, pandas==0.16.0, pandocfilters==1.2.4, pep8==1.6.2, pexpect==3.3, pockets==0.2.4, psutil==2.2.1, pycurl==7.19.5.1, pyflakes==0.8.1, pyparsing==2.0.3, pyqtgraph==0.9.10, python-dateutil==2.4.2, python-igraph==0.7.1-4, pytz==2015.2, pyzmq==14.5.0, requests==2.6.0, rope-py3k==0.9.4-1, rpy2==2.5.6, scikit-learn==0.16.0, scipy==0.15.1, six==1.9.0, snowbal

In [3]:
from spgr import lg

## Propagation of Spindles

In [22]:
from spgr.constants import GROUP_PATH, HEMI_SUBJ, SPINDLE_OPTIONS, DATA_OPTIONS, CHAN_TYPE
from spgr.read_data import get_data
from spgr.detect_spindles import get_spindles
from spgr.stats_on_spindles import create_spindle_groups

In [9]:
images_dir = GROUP_PATH.joinpath('images').joinpath('propagation')
if not images_dir.exists():
    images_dir.mkdir()

PLOT_COLOR = 'kw'

In [24]:
subj = 'EM09'
REREF = 'avg'
data = get_data(subj, 'sleep', CHAN_TYPE, reref=REREF, **DATA_OPTIONS)
spindles = get_spindles(subj, reref=REREF, **SPINDLE_OPTIONS)



In [28]:
sp = spindles.spindle[0]
sp

{'area_under_curve': 55.787234105316898,
 'chan': 'GR58',
 'end_time': 42661.976819898424,
 'peak_freq': 12.668041237113403,
 'peak_time': 42661.425966922776,
 'peak_val': 44.522812608100409,
 'start_time': 42660.085948691238}

In [17]:
spindle_window = 1

In [18]:
start_time = sorted([sp['start_time'] for sp in spindles.spindle])[0] - spindle_window
end_time = sorted([sp['end_time'] for sp in spindles.spindle])[-1] + spindle_window

In [25]:
from phypno.trans import Select

In [39]:
def find_trial_with_spindle(data, trial_time):
    for i, trial_time in enumerate(data.axis['time']):
        if sp['peak_time'] >= trial_time[0] and sp['peak_time'] <= trial_time[-1]:
            return i

In [65]:
sel = Select(trial=(find_trial_with_spindle(data, sp['peak_time']), ), time=(sp['peak_time'] - spindle_window, sp['peak_time'] + spindle_window + 1 / data.s_freq))

In [66]:
sp_data = sel(data)

#### TODO: filter in spindle band

In [67]:
dat = sp_data(trial=0)
ref_dat = sp_data(trial=0, chan=sp['chan'])

In [68]:
from numpy import correlate

In [69]:
one_chan = dat[0]
c = correlate(one_chan, ref_dat, 'same')

In [70]:
from pyqtgraph import plot

In [80]:
plot(t, c)

In [90]:
from numpy import *

In [78]:
t = arange(-spindle_window, spindle_window, 1/data.s_freq)

In [81]:
plot(one_chan)

In [82]:
plot(ref_dat)

In [83]:
c = correlate(ref_dat, one_chan, 'same')
plot(t, c)

In [84]:
from scipy.signal import hamming

In [92]:
hamm = hamming(512)

In [97]:
h0 = r_[hamm[450:], hamm[:450]]

In [98]:
c = correlate(ref_dat, one_chan, 'same')


In [99]:
plot(h0)
plot(hamm)

In [108]:
c = correlate(hamm, hamm, 'same')

In [109]:
plot(t, c)

In [106]:
mean(c[:256]) - mean(c[256:])

63.082022972161482

In [111]:
from numpy.linalg import norm

In [112]:
norm(hamm)

14.250536831993383

In [113]:
sum(c)

70320.408224734507

In [ ]:
no